In [6]:
import pandas as pd 
from requests_html import HTMLSession
import os
import requests
from lxml import html 
import csv
from datetime import date,datetime
import re 
from bs4 import BeautifulSoup
import nums_from_string

In [7]:
def fetchSession(url):
    session = HTMLSession()
    r = session.get(url)
    return r

def getTrades(r):
    table = r.html.find('table')[0]
    rows = table.find('tr')
    return rows[1:]

In [8]:
def getHTML(url):
    r = fetchSession(url)
    h = r.text
    doc = BeautifulSoup(h, 'html.parser')
    return doc

def parseToMillions(value_string):
    unit = value_string[-1:]
    number = nums_from_string.get_nums(value_string)[0]
    #keep in units of millions
    if unit == 'B':
        number = number * 1000
    elif unit == 'T':
        number = number * 1000000
    return number

In [9]:
# first row of second table on yahoo finance site 
def getFirstRowEntry(ticker):
    url = 'https://finance.yahoo.com/quote/{}/'.format(ticker)
    soup = getHTML(url)
    quote_summary = soup.find(id='quote-summary')
    if quote_summary is None:
        return -1
    tables = quote_summary.find_all('table')
    if len(tables) == 0:
        return -1
    # right side table
    mc_table = tables[1]
    # get all rows
    mc_rows = mc_table.find_all('td')
    # entire row 
    mc_string = str(mc_rows[1])
    return mc_string

In [10]:
def isStock(row_one):
    flag = 'data-test="(.*)-value'
    marker = re.search(
        flag, row_one
    ).group(1)
    if marker == 'MARKET_CAP':
        return 1
    elif marker == 'NET_ASSETS':
        return 0
    # N/A
    else:
        return -1

In [14]:
def getNAVCAP(row_one):
    value = re.search('>(.*)<', row_one).group(1)
    if value == 'N/A':
        return -1
    return round(parseToMillions(value),2)

In [16]:
aapl_row = getFirstRowEntry('AAPL')
vug_row = getFirstRowEntry('VUG')
print(isStock(aapl_row))
print(isStock(vug_row))
print(getNAVCAP(aapl_row))
print(getNAVCAP(vug_row))

1
0
2722000.0
169450.0


In [17]:
def isSmallCapStock(ticker):
    row_output = getFirstRowEntry(ticker)
    if isStock(row_output) == 1:
            mkt_cap = getNAVCAP(row_output)
            if mkt_cap < 2000 and mkt_cap > 0:
                return True
    return False

In [21]:
print(isSmallCapStock('ECOM'))

True
